This notebook is intended to process complete datasets of each pitchers pitches.This is different from the other notebook because it does not append previous pitch data to each pitch. Furthermore, the data does not remove post pitch information for the pitch on the row, meaning the release speed, pfx_x, etc. correspond to the pitch on that row.

In [27]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
verlander = pd.read_csv('Data/raw_data/verlander.csv')
scherzer = pd.read_csv('Data/raw_data/scherzer.csv')
lester =  pd.read_csv('Data/raw_data/lester.csv')
hamels =  pd.read_csv('Data/raw_data/hamels.csv')

In [28]:
pitchers = [lester,verlander,scherzer,hamels]
for pitcher in pitchers:
    for index,row in pitcher.iterrows():
        if row["pitch_type"] == 'IN'\
        or row["pitch_type"] == 'PO'\
        or row["pitch_type"] == 'UN'\
        or (row["pitch_type"] == 'SL' and row["player_name"] == "Cole Hamels")\
        or (row["pitch_type"] == 'SL' and row["player_name"] == "Jon Lester")\
        or (row["pitch_type"] == 'FC' and row["player_name"] == "Justin Verlander")\
        or (row["pitch_type"] == 'FT' and row["player_name"] == "Justin Verlander")\
        or (row["pitch_type"] == 'FT' and row["player_name"] == "Max Scherzer")\
        or row["pitch_type"] != row["pitch_type"]:
            pitcher.drop(index,inplace=True)
hamels.reset_index(inplace=True)
verlander.reset_index(inplace=True)
scherzer.reset_index(inplace=True)
lester.reset_index(inplace=True)

In [29]:
for pitcher in pitchers:
    pitcher.drop(['spin_dir', 'spin_rate_deprecated','break_angle_deprecated', 'break_length_deprecated',
                 'game_type','inning_topbot', 'hc_x', 'hc_y',
                 'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
                 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
                 'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
                 'release_spin_rate', 'release_extension',
                 'pitcher.1','fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
                 'fielder_7', 'fielder_8', 'fielder_9',
                 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle','launch_speed_angle',
                 'pitch_name','if_fielding_alignment', 'of_fielding_alignment'],axis=1,inplace=True)

In [30]:
for pitcher in pitchers:
    pitcher.drop([ 'events','des',
                   'type', 'hit_location', 'bb_type', 'woba_value',
                   'woba_denom', 'babip_value', 'iso_value','game_date','player_name','index'],axis=1,inplace=True)

In [31]:
for pitcher in pitchers:
    pitcher['on_1b'] = pitcher['on_1b'].apply(lambda row: 1 if row == row else 0)
    pitcher['on_2b'] = pitcher['on_2b'].apply(lambda row: 1 if row == row else 0)
    pitcher['on_3b'] = pitcher['on_3b'].apply(lambda row: 1 if row == row else 0)
    pitcher['stand'] = pitcher['stand'].apply(lambda row: 1 if row == 'L' else 0)
    pitcher['p_throws'] = pitcher['p_throws'].apply(lambda row: 1 if row == 'L' else 0)
    pitcher.rename(columns={'stand':'stand_L','p_throws':'throws_L'},inplace=True)

In [32]:
#Verlander:
##DET(2008-2017) HOU(2017-2019)
def verlander_home(verlander):
    if verlander['home_team'] == 'DET':
        if verlander['game_year'] in [2008,2009,2010,2011,2012,2013,2014,2015,2016,2017]:
                return 1
        else: return 0 
    elif verlander['home_team'] == 'HOU':
        if verlander['away_team'] == 'DET':
            if verlander['game_year'] == 2017:
                return 0
            else: return 0 
        elif verlander['game_year'] in [2017,2018,2019]:
            return 1    
        else: return 0   
    else: return 0 

In [33]:
#Lester:
## BOS(2008-2014) OAK(2014) CHC(2015-2019)
def lester_home(lester):
    if lester['home_team'] == 'BOS':
        if lester['game_year'] in [2008,2009,2010,2011,2012,2013,2014]:
            return 1
        else: return 0
    elif lester['home_team'] == 'CHC':
        if lester['game_year'] in [2015,2016,2017,2018,2019]:
            return 1
        else:
            return 0
    elif lester['home_team'] == 'OAK':
        if lester['away_team'] == 'BOS':
            if lester['game_year'] == 2014:
                return 0
            else:
                return 0
        elif lester['game_year'] == 2014:
            return 1
        else: return 0
    else:
        return 0

In [34]:
#Scherzer
## ARI(2008-2009) DET(2010-2014) WAS(2015-2019)
def scherzer_home(scherzer):
    if scherzer['home_team'] == 'ARI':
        if scherzer['game_year'] in [2008,2009]:
            return 1
        else: return 0
    elif scherzer['home_team'] == 'DET':
        if scherzer['game_year'] in [2010,2011,2012,2013,2014]:
            return 1
        else: return 0
    elif scherzer['home_team'] == 'WAS':
        if scherzer['game_year'] in [2015,2016,2017,2018,2019]:
            return 1 
        else: return 0
    else: return 0

In [35]:
#Hamels:
##PHI(2008-2015) TEX(2015-2018) CHC(2018-2019) 
def hamels_home(hamels):
    if hamels['home_team'] == 'PHI':
        if hamels['game_year'] in [2008,2009,2010,2011,2012,2013,2014,2015]:
            return 1
        else: return 0
    elif hamels['home_team'] == 'TEX':
        if hamels['game_year'] in [2015,2016,2017,2018]:
            return 1
        else: return 0
    elif hamels['home_team'] == 'CHC':
        if hamels['game_year'] in [2018,2019]:
            return 1
        else: return 0
    else: return 0

In [36]:
verlander['home_team'] = verlander.apply(lambda row: verlander_home(row),axis=1)
lester['home_team'] = lester.apply(lambda row: lester_home(row),axis=1)
hamels['home_team'] = hamels.apply(lambda row: hamels_home(row),axis=1)
scherzer['home_team'] = scherzer.apply(lambda row: scherzer_home(row),axis=1)

In [37]:
pitch_ohe = pd.get_dummies(verlander['pitch_type'])
description_ohe = pd.get_dummies(verlander['description'])
verlander.drop(['pitch_type','description'],axis=1,inplace=True)
verlander = verlander.join(pitch_ohe)
verlander =verlander.join(description_ohe)

pitch_ohe = pd.get_dummies(hamels['pitch_type'])
description_ohe = pd.get_dummies(hamels['description'])
hamels.drop(['pitch_type','description'],axis=1,inplace=True)
hamels = hamels.join(pitch_ohe)
hamels = hamels.join(description_ohe)

pitch_ohe = pd.get_dummies(scherzer['pitch_type'])
description_ohe = pd.get_dummies(scherzer['description'])
scherzer.drop(['pitch_type','description'],axis=1,inplace=True)
scherzer = scherzer.join(pitch_ohe)
scherzer = scherzer.join(description_ohe)

pitch_ohe = pd.get_dummies(lester['pitch_type'])
description_ohe = pd.get_dummies(lester['description'])
lester.drop(['pitch_type','description'],axis=1,inplace=True)
lester = lester.join(pitch_ohe)
lester = lester.join(description_ohe)

In [38]:
for pitcher in pitchers:
    pitcher.drop(['home_score','away_score','post_away_score','post_home_score','post_bat_score','post_fld_score'],axis=1,inplace=True)

In [39]:
ver_cols = ['CH','CU','FF','SL', 'stand_L', 'home_team', 'balls',
       'strikes', 'game_year', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up',
       'inning', 'at_bat_number', 'pitch_number', 'bat_score', 'fld_score','ball', 'blocked_ball',
       'called_strike', 'foul', 'foul_bunt', 'foul_tip', 'hit_by_pitch',
       'hit_into_play', 'hit_into_play_no_out', 'hit_into_play_score',
       'missed_bunt', 'swinging_strike', 'swinging_strike_blocked',
       'release_speed', 'release_pos_x', 'release_pos_y', 'release_pos_z', 'zone', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z']
verlander = verlander[ver_cols]

In [40]:
scher_cols = ['CH', 'CU', 'FC', 'FF','SL', 'stand_L', 'home_team', 'balls',
       'strikes', 'game_year', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up',
       'inning', 'at_bat_number', 'pitch_number', 'bat_score', 'fld_score','ball', 'blocked_ball',
       'called_strike', 'foul', 'foul_bunt', 'foul_tip', 'hit_by_pitch',
       'hit_into_play', 'hit_into_play_no_out', 'hit_into_play_score',
       'missed_bunt', 'swinging_strike', 'swinging_strike_blocked',
       'release_speed', 'release_pos_x', 'release_pos_y', 'release_pos_z', 'zone', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z']
scherzer = scherzer[scher_cols]

In [41]:
hamels_cols = ['CH', 'CU', 'FC', 'FF','FT', 'stand_L', 'home_team', 'balls',
       'strikes', 'game_year', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up',
       'inning', 'at_bat_number', 'pitch_number', 'bat_score', 'fld_score', 'ball', 'blocked_ball',
       'called_strike', 'foul', 'foul_bunt', 'foul_tip', 'hit_by_pitch',
       'hit_into_play', 'hit_into_play_no_out', 'hit_into_play_score',
       'missed_bunt', 'swinging_strike', 'swinging_strike_blocked',
       'release_speed', 'release_pos_x', 'release_pos_y', 'release_pos_z', 'zone', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z']
hamels = hamels[hamels_cols]

In [42]:
lester_cols = ['CH', 'CU', 'FC', 'FF','SI', 'stand_L', 'home_team', 'balls',
       'strikes', 'game_year', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up',
       'inning', 'at_bat_number', 'pitch_number', 'bat_score', 'fld_score', 'ball', 'blocked_ball',
       'called_strike', 'foul', 'foul_bunt', 'foul_tip', 'hit_by_pitch',
       'hit_into_play', 'hit_into_play_no_out', 'hit_into_play_score',
       'missed_bunt', 'swinging_strike', 'swinging_strike_blocked',
       'release_speed', 'release_pos_x', 'release_pos_y', 'release_pos_z', 'zone', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z']
lester = lester[lester_cols]

In [49]:
pitchers = [verlander,scherzer,lester,hamels]
for pitcher in pitchers:
    for column in pitcher.columns:
        print("column:",column)
        print("nas:",sum(pitcher[column] != pitcher[column] ))

column: CH
nas: 0
column: CU
nas: 0
column: FF
nas: 0
column: SL
nas: 0
column: stand_L
nas: 0
column: home_team
nas: 0
column: balls
nas: 0
column: strikes
nas: 0
column: on_3b
nas: 0
column: on_2b
nas: 0
column: on_1b
nas: 0
column: outs_when_up
nas: 0
column: inning
nas: 0
column: at_bat_number
nas: 0
column: pitch_number
nas: 0
column: bat_score
nas: 0
column: fld_score
nas: 0
column: ball
nas: 0
column: blocked_ball
nas: 0
column: called_strike
nas: 0
column: foul
nas: 0
column: foul_bunt
nas: 0
column: foul_tip
nas: 0
column: hit_by_pitch
nas: 0
column: hit_into_play
nas: 0
column: hit_into_play_no_out
nas: 0
column: hit_into_play_score
nas: 0
column: missed_bunt
nas: 0
column: swinging_strike
nas: 0
column: swinging_strike_blocked
nas: 0
column: release_speed
nas: 0
column: zone
nas: 0
column: pfx_x
nas: 0
column: pfx_z
nas: 0
column: plate_x
nas: 0
column: plate_z
nas: 0
column: CH
nas: 0
column: CU
nas: 0
column: FC
nas: 0
column: FF
nas: 0
column: SL
nas: 0
column: stand_L
na

In [44]:
for pitcher in pitchers:
    for index,row in pitcher.iterrows():
        if row["pfx_x"] != row["pfx_x"]\
        or row["pfx_z"] != row["pfx_z"]\
        or row["plate_x"] != row["plate_x"]\
        or row["plate_z"] != row["plate_z"]:
            pitcher.drop(index,inplace=True)
hamels.reset_index(inplace=True)
verlander.reset_index(inplace=True)
scherzer.reset_index(inplace=True)
lester.reset_index(inplace=True)

In [45]:
pitchers = [verlander,lester,hamels,scherzer]
for pitcher in pitchers:
    pitcher.drop(['release_pos_x','release_pos_y','release_pos_z'],axis=1,inplace=True)

In [46]:
for pitcher in pitchers:
    pitcher.drop(['game_year'],axis=1,inplace=True)

In [47]:
for pitcher in pitchers:
    pitcher.drop(['index'],axis=1,inplace=True)

In [48]:
verlander.to_csv('Data/full_data/verlander_full.csv')
lester.to_csv('Data/full_data/lester_full.csv')
hamels.to_csv('Data/full_data/hamels_full.csv')
scherzer.to_csv('Data/full_data/scherzer_full.csv')